## 卷积操作和卷积层的定义

In [5]:
import torch
from torch import nn
import d2l


def corr2d(X, K):
    """
    计算二维互相关运算
    :param X: 要计算的图像
    :param K: 卷积核
    :return:
    """
    height, width = K.shape
    new_size = (X.shape[0] - height + 1, X.shape[1] - width + 1)
    Y = torch.zeros(new_size)
    for i in range(new_size[0]):
        for j in range(new_size[1]):
            Y[i, j] = (X[i:i + height, j:j + width] * K).sum()
    return Y


X = torch.tensor([
    [0.0, 1.0, 2.0],
    [3.0, 4.0, 5.0],
    [6.0, 7.0, 8.0]
])
kernel = torch.tensor([
    [0.0, 1.0],
    [2.0, 3.0]
])

corr2d(X, kernel)

tensor([[19., 25.],
        [37., 43.]])

In [6]:
class Conv2d(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        """计算卷积的操作"""
        return corr2d(x, self.weight) + self.bias

## 利用1*2卷积核对图像进行边缘检测

使用X模拟图像

构造卷积核 高度为1 宽度为2 !!!注意 卷积核一定是tensor 而非向量
进行互运算 如果两个元素相同 输出为0，否则输出1

In [28]:
X = torch.ones((6, 8))

# X[:, 2:6] = 0
# K_vert = torch.tensor([[1.0, -1.0]])  # 这里的卷积核只可以判断垂直边缘

X[2:4, :] = 0
K_horiz = torch.tensor([[1.0],
                        [-1.0]])  # 这里的卷积核只可以判断水平边缘
result = corr2d(X, K_horiz).abs()
X, result

(tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.]]),
 tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]))

In [29]:
edge = []
for i in range(result.shape[0]):
    for j in range(result.shape[1]):
        if result[i, j] == 1.0:
            edge.append((i, j))
edge

[(1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7)]